In [1]:
import json
import numpy as np

def fit_function(x, a, ref, shift=0):
    return a*(x + ref)**2 + shift

def loss(params, x_data, y_data):
    a, ref, shift = params
    y_pred = fit_function(x_data, a, ref, shift)
    return np.sum(np.abs(np.floor(y_pred) - np.floor(y_data)))

def time_to_seconds(time_str):
    parts = time_str.split(':')
    seconds = float(parts[-1])
    for i, part in enumerate(reversed(parts[:-1])):
        seconds += int(part) * 60 ** (i+1)
    return seconds

points_table = json.load(open("2025_points_table.json", "r"))
points = np.array(points_table["Points"], dtype=float) + 0.5

def get_fit(gender, event):
    times = np.array(points_table[gender][event]['Scores'])
    times = np.array([time_to_seconds(t) if t != "-" else np.nan for t in times])
    print(f"{event}: {times[~np.isnan(times)][-1]}")
    # curve fitting
    x_data = times[~np.isnan(times)]
    y_data = points[~np.isnan(times)]
    a, b, c = np.polyfit(x_data, y_data, 2)
    ref = b / (2 * a)
    shift = c - b**2 / (4 * a)
    # calculate the loss
    error = loss([a, ref, shift], x_data, y_data)
    print(f"{event}: a={a}, ref={ref}, shift={shift}, error={error}")
    return a, ref, shift

In [4]:
fit_params = {"Men": {},
              "Women": {}}

for gender in fit_params.keys():
    for event in points_table[gender].keys():
        fits = get_fit(gender, event)
        fit_params[gender][event] = {}
        fit_params[gender][event]["Fits"] = fits
        fit_params[gender][event]["Distance"] = points_table[gender][event]['Distance']
        fit_params[gender][event]["Category"] = points_table[gender][event]['Category']
        fit_params[gender][event]["Surface"] = points_table[gender][event]['Surface']
        fit_params[gender][event]["Mixed"] = points_table[gender][event]['Mixed']

# save the fit parameters to a JSON file
with open("2025_fit_params.json", "w") as f:
    json.dump(fit_params, f, indent=4)


50m: 9.09
50m: a=95.82235386298211, ref=-9.199401087362443, shift=0.1234021683494575, error=6.0
55m: 9.88
55m: a=78.92276088460508, ref=-9.999168296259692, shift=0.17374067832133733, error=5.0
60m: 10.57
60m: a=68.6203220046643, ref=-10.699280016351242, shift=0.1405724318110515, error=12.0
100m: 16.79
100m: a=24.642211663306004, ref=-16.997531558274364, shift=0.28133795201574685, error=12.0
200m: 35.05
200m: a=5.083329625707267, ref=-35.49111159736059, shift=0.5957410214887204, error=12.0
200m sh: 35.55
200m sh: a=5.042898433343424, ref=-35.99171911026435, shift=0.5826109375320812, error=22.0
300m: 56.46
300m: a=1.8296570247435697, ref=-57.19768814338232, shift=0.4446414074582208, error=9.0
300m sh: 57.25
300m sh: a=1.802963929759478, ref=-57.99543642591087, shift=0.48213317582485615, error=0.0
400m: 78.01
400m: a=1.021013042553312, ref=-78.99469305639144, shift=0.5029880051670261, error=0.0
400m sh: 79.59
400m sh: a=0.9810285010269417, ref=-80.5944655442887, shift=0.5063326026674986, 